In [2]:
#Importamos la librerías y variables necesarias
import mysql.connector
import sqlite3
import pandas as pd
from flask import Flask, jsonify, request
import pickle
import os
#################################################################################
#################################################################################
"""Definimos las variables generales para atacar la base de datos"""
#Definimos un cursor con el que atacar a la DB
cnx = mysql.connector.connect(
    user="admin",
    password="admin123",
    host="test-db.cze2nnbbx5pc.eu-west-3.rds.amazonaws.com",
    database="prueba")
cursor = cnx.cursor()
#Definimos una función mediante la que ejecutar las querys devolviendo un dataframe
def make_query(code):
    cursor.execute(code)
    results = cursor.fetchall()
    cnx.commit()
    return(results)
#Definimos una función mediante la que ejecutar las querys devolviendo un dataframe
def make_query_dataframe(code):
    cursor.execute(code)
    results = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]  # Obtener los nombres de las columnas
    df = pd.DataFrame(results, columns=column_names)  # Crear el DataFrame
    return df

In [15]:
datos_raw=pd.read_csv("hr_names.csv", index_col="EmployeeID")

In [16]:
datos_raw.columns

Index(['hrs', 'JobInvolvement', 'PerformanceRating', 'EnvironmentSatisfaction',
       'JobSatisfaction', 'WorkLifeBalance', 'Age', 'Attrition',
       'BusinessTravel', 'Department', 'DistanceFromHome', 'Education',
       'EducationField', 'EmployeeCount', 'Gender', 'JobLevel', 'JobRole',
       'MaritalStatus', 'MonthlyIncome', 'NumCompaniesWorked', 'Over18',
       'PercentSalaryHike', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'YearsAtCompany',
       'YearsSinceLastPromotion', 'YearsWithCurrManager', 'Name'],
      dtype='object')

In [17]:

#borrar employee_count
#borrar over 18
#borrar standar hours

In [18]:
data=datos_raw.drop(columns=["EmployeeCount","Over18","StandardHours"])

In [19]:
data.columns

Index(['hrs', 'JobInvolvement', 'PerformanceRating', 'EnvironmentSatisfaction',
       'JobSatisfaction', 'WorkLifeBalance', 'Age', 'Attrition',
       'BusinessTravel', 'Department', 'DistanceFromHome', 'Education',
       'EducationField', 'Gender', 'JobLevel', 'JobRole', 'MaritalStatus',
       'MonthlyIncome', 'NumCompaniesWorked', 'PercentSalaryHike',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
       'Name'],
      dtype='object')

In [20]:
order=['Name','hrs', 'JobInvolvement', 'PerformanceRating', 'EnvironmentSatisfaction',
       'JobSatisfaction', 'WorkLifeBalance', 'Age', 'Attrition',
       'BusinessTravel', 'Department', 'DistanceFromHome', 'Education',
       'EducationField', 'Gender', 'JobLevel', 'JobRole', 'MaritalStatus',
       'MonthlyIncome', 'NumCompaniesWorked', 'PercentSalaryHike',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager'
       ]

In [21]:
data = data.reindex(columns=order)

In [22]:
data

,Name,hrs,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,Attrition,BusinessTravel,...,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
EmployeeID,,,,,,,,,,,,,,,,,,,,,
1,Jennifer Grant,7.315971,High,Excellent,High,Very High,Good,51,No,Travel_Rarely,...,Married,131160,1.0,11,0,1.0,6,1,0,0
2,Brittany Dunn,6.816081,Medium,Outstanding,High,Medium,Best,31,Yes,Travel_Frequently,...,Single,41890,0.0,23,1,6.0,3,5,1,4
3,Joel Rodgers,6.789215,High,Excellent,Medium,Medium,Bad,32,No,Travel_Frequently,...,Married,193280,1.0,15,3,5.0,2,5,0,3
4,Gabriel Henson PhD,7.877561,Medium,Excellent,Very High,Very High,Better,38,No,Non-Travel,...,Married,83210,3.0,11,3,13.0,5,8,7,5
5,James Bryant,10.275802,High,Excellent,Very High,Low,Better,32,No,Travel_Rarely,...,Single,23420,4.0,12,2,9.0,2,6,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,Rebecca Carlson,8.316921,Medium,Excellent,High,Very High,Better,29,No,Travel_Rarely,...,Single,35390,1.0,18,0,6.0,2,6,1,5
4406,Kimberly Tucker,5.897197,High,Excellent,Very High,Low,Better,42,No,Travel_Rarely,...,Single,60290,3.0,17,1,10.0,5,3,0,2
4407,Warren Phillips,7.149526,Medium,Excellent,Very High,Very High,Better,29,No,Travel_Rarely,...,Divorced,26790,2.0,15,0,10.0,2,3,0,2


In [23]:
make_query("""CREATE TABLE employee_raw (
    id_employee INT NOT NULL,
    name VARCHAR(50),
    hours DECIMAL(4,2),
    involvement VARCHAR(50),
    performance VARCHAR(50),
    environment VARCHAR(50),
    satisfaction VARCHAR(50),
    life_balance VARCHAR(50),
    age INT,
    attrition VARCHAR(50),
    travel VARCHAR(50),
    department VARCHAR(50),
    distance_home INT,
    education VARCHAR(50),
    education_field VARCHAR(50),
    gender VARCHAR(50),
    job_level INT,
    role VARCHAR(50),
    marital_status VARCHAR(50),
    income INT,
    previous_companies INT,
    salary_hike INT,
    stock_option INT,
    total_working_years INT,
    training_ly INT,
    years_company INT,
    last_promotion INT,
    years_curr_manager INT,
    PRIMARY KEY (id_employee))"""
)


[]

In [24]:
data=data.reset_index()

In [25]:
# Cargar los datos del CSV a la tabla
for row in data.itertuples(index=False):
    values = ', '.join("'" + str(value).replace("'", "''") + "'" for value in row)
    query = f"INSERT INTO employee_raw VALUES ({values})"
    cursor.execute(query)
    print(row)
# Confirmar los cambios en la base de datos
cnx.commit()
# Cerrar el cursor y la conexión
cursor.close()
cnx.close()

Pandas(EmployeeID=1, Name='Jennifer Grant', hrs=7.315970548861944, JobInvolvement='High', PerformanceRating='Excellent', EnvironmentSatisfaction='High', JobSatisfaction='Very High', WorkLifeBalance='Good', Age=51, Attrition='No', BusinessTravel='Travel_Rarely', Department='Sales', DistanceFromHome=6, Education='College', EducationField='Life Sciences', Gender='Female', JobLevel=1, JobRole='Healthcare Representative', MaritalStatus='Married', MonthlyIncome=131160, NumCompaniesWorked=1.0, PercentSalaryHike=11, StockOptionLevel=0, TotalWorkingYears=1.0, TrainingTimesLastYear=6, YearsAtCompany=1, YearsSinceLastPromotion=0, YearsWithCurrManager=0)
Pandas(EmployeeID=2, Name='Brittany Dunn', hrs=6.816080990629167, JobInvolvement='Medium', PerformanceRating='Outstanding', EnvironmentSatisfaction='High', JobSatisfaction='Medium', WorkLifeBalance='Best', Age=31, Attrition='Yes', BusinessTravel='Travel_Frequently', Department='Research & Development', DistanceFromHome=10, Education='Below College